**Code Cell 1** - Importing required libraries and data

In [ ]:
!pip install s3fs==2023.9.2 
!pip install boto3
# importing the dataset
import pandas as pd
import s3fs

fs=s3fs.S3FileSystem(anon=False) # initialize the file system

s3_url='s3://amazon-sagemaker- # location of the file in s3 bucket

data=pd.read_csv(s3_url)

data.head()

  Using cached botocore-1.31.17-py3-none-any.whl.metadata (5.9 kB)
Using cached botocore-1.31.17-py3-none-any.whl (11.1 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.162
    Uninstalling botocore-1.34.162:
      Successfully uninstalled botocore-1.34.162
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-sagemaker-sql-editor 0.1.15 requires aiobotocore<3,>=2.7.0, but you have aiobotocore 2.5.4 which is incompatible.
amazon-sagemaker-sql-editor 0.1.15 requires botocore<2,>=1.31.64, but you have botocore 1.31.17 which is incompatible.
boto3 1.34.162 requires botocore<1.35.0,>=1.34.162, but you have botocore 1.31.17 which is incompatible.
s3transfer 0.10.4 requires botocore<2.0a.0,>=1.33.2, but you have botocore 1.31.17 which is incompatible.
sagemaker-studio 1.0.9 requires botocore>=1.34.106, but you have botocore 1.31.17

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
0,IVS7do_HBzroiCiymNdxDg,fdFgZQQYQJeEAshH4lxSfQ,sGy67CpJctjeCWClWqonjA,3,1,1,0,"OK, the hype about having Hatch chili in your ...",1/27/2020 22:59,1
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream.... onl...,4/19/2020 5:33,1
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2/29/2020 19:43,1
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,I've gone to claim Jumpers all over the US and...,3/14/2020 21:47,1
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,"If you haven't been to Maynard's kitchen, it'...",1/17/2020 20:32,1


**Code cell 2**: Data Preprocessing

In [28]:
# Filter positive and negative reviews
positive_reviews = data[data['Sentiment'] == 1].sample(n=50, random_state=42)
negative_reviews = data[data['Sentiment'] == 0].sample(n=50, random_state=42)

# Combine them into a new dataset
sampled_df = pd.concat([positive_reviews, negative_reviews])

# Add a new column with labeled sentiment
sampled_df['Sentiment_Label'] = sampled_df['Sentiment'].map({1: 'Positive', 0: 'Negative'})

# Select only the 'text' and 'Sentiment_Label' columns
sampled_df = sampled_df[['text', 'Sentiment_Label']]

# Shuffle the dataset
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the new dataset
sampled_df.to_csv('sampled_reviews.csv', index=False)

#checking the new dataset
print(len(sampled_df))
# Display the first few rows
print(sampled_df.head())


100
                                                text Sentiment_Label
0  The customer service here is garbage. Tried pl...        Negative
1  I can only offer three stars due to the loud t...        Negative
2  We got the Plantano y Fritos y Frijoles and th...        Negative
3  Restaurants these days have lowered their stan...        Positive
4  Came here for their Fat Tuesday/Mardi Gras cel...        Positive


In [29]:
import random
import json
import boto3

# Creating a bedrock runtime client.
client = boto3.client("bedrock-runtime", region_name="us-east-1") 

# each model has a unique Model Identifier
model_id='anthropic.claude-3-sonnet-20240229-v1:0' # no of tokens for input
# import a data sample from the dataset and prompt the model to perform sentiment analysis.

random_index=random.randint(0,100) # create a random index

data_sample=sampled_df.iloc[random_index]# access a random sample from the dataset using random_index

data_sample.head()

# Sample data extraction
review_d = sampled_df['text'].iloc[random.randint(0, len(sampled_df)-1)]  # Randomly select a review
sentiment = sampled_df['Sentiment_Label'].iloc[random.randint(0, len(sampled_df)-1)]  # Actual sentiment label

# Define the prompt to send to Claude for sentiment analysis
prompt = f'''
Perform sentiment analysis on the given review, only predict whether it is positive, neutral, or negative.
{review_d}
'''

# Prepare the API request for Anthropic Claude
native_request = {
    "anthropic_version": "bedrock-2023-05-31",  # Version of the Claude model
    "max_tokens": 25,  # Max tokens for response
    "temperature": 0.4,  # Set the creativity of the model
    "messages": [
        {"role": "user", "content": prompt}  # User message with the prompt
    ]
}

# Convert the request to JSON format
request = json.dumps(native_request)

# Invoke the model through the AWS Bedrock client
response = client.invoke_model(modelId=model_id, body=request)

# Parse the model's response
model_response = json.loads(response["body"].read())  # Extract the response body
prediction = model_response.get("content", [""])[0].get("text", "")  # Extract the predicted sentiment

# Display the predicted and actual sentiment
print("---------------Predicted Sentiment (From LLM)-----------------------")
print("--------------------------------------------------------------------\n")
print(prediction)

print("\n---------------Actual Sentiment (From Dataset)----------------------")
print("---------------------------------------------------------------------\n")
print(sentiment)

---------------Predicted Sentiment (From LLM)-----------------------
--------------------------------------------------------------------

The sentiment of the given review is negative.

The reviewer expresses dissatisfaction and frustration with several aspects of their

---------------Actual Sentiment (From Dataset)----------------------
---------------------------------------------------------------------

Negative


**Code cell 3**: Perform Sentiment Analysis Using Zero-Shot Learning

In [61]:
import json
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import re
import json

def predict_sentiment(sampled_df, model_id, client):
    predictions = []
    actual = []
    skipped_reviews = 0  # Counter for skipped reviews
    misclassified_reviews = []  # List to store misclassified reviews

    print(f"Total reviews in dataset: {sampled_df.shape[0]}")

    # Loop through the DataFrame and perform sentiment prediction for each review
    for index, row in tqdm(sampled_df.iterrows(), total=sampled_df.shape[0], desc="Predicting Sentiments"):
        review = row['text']
        actual_sentiment = row['Sentiment_Label'].strip().lower()  # Ensure actual sentiment is lowercase
        
        # Skip rows where sentiment is not 'positive' or 'negative'
        if actual_sentiment not in ['positive', 'negative']:
            print(f"Skipping index {index}: Sentiment '{actual_sentiment}' is not valid.")
            skipped_reviews += 1
            continue  # Ensure this is inside the loop

        # Improved prompt
        prompt = f"""Please predict the sentiment of the following review using only 'positive' or 'negative' as your response.
        
        Review: {review}
        
        Your response should be only one word: 'positive' or 'negative'.
        """
        
        native_request = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 5,  # Reduce token limit to avoid long responses
            "temperature": 0.3,  # Lower temperature for more deterministic output
            "messages": [
                {"role": "user", "content": prompt}
            ]
        }
        
        request = json.dumps(native_request)

        try:
            # Send the request to the model and receive the response
            response = client.invoke_model(modelId=model_id, body=request)
            response_body = response["body"].read()

            if response_body:
                model_response = json.loads(response_body)
                
                # Extract the sentiment prediction from the model response
                raw_prediction = model_response.get("content", [""])[0].get("text", "").strip().lower()

                # Use regex to extract 'positive' or 'negative' from response
                match = re.search(r"\b(positive|negative)\b", raw_prediction)
                prediction = match.group(1) if match else None

                # If extraction fails, re-prompt the model with a stricter query
                if not prediction:
                    print(f"Re-prompting for index {index}: Model response '{raw_prediction}' did not contain 'positive' or 'negative'.")
                    
                    strict_prompt = f"""Please provide only one word: 'positive' or 'negative' for this review:
                    
                    Review: {review}
                    
                    Only reply with 'positive' or 'negative'. No additional text."""
                    
                    native_request["messages"][0]["content"] = strict_prompt  # Modify the prompt
                    strict_request = json.dumps(native_request)

                    response = client.invoke_model(modelId=model_id, body=strict_request)
                    response_body = response["body"].read()

                    if response_body:
                        model_response = json.loads(response_body)
                        raw_prediction = model_response.get("content", [""])[0].get("text", "").strip().lower()
                        match = re.search(r"\b(positive|negative)\b", raw_prediction)
                        prediction = match.group(1) if match else None

                # If re-prompting also fails, skip the review
                if not prediction:
                    print(f"Skipping index {index}: Even after re-prompting, model response '{raw_prediction}' did not contain 'positive' or 'negative'.")
                    skipped_reviews += 1
                    continue
                
                # Store valid predictions
                predictions.append(prediction)
                actual.append(actual_sentiment)

                # If prediction doesn't match actual sentiment, add to misclassified list
                if prediction != actual_sentiment:
                    misclassified_reviews.append({
                        'index': index,
                        'review': review,
                        'predicted_label': prediction,
                        'actual_label': actual_sentiment
                    })

        except Exception as e:
            print(f"Error during model invocation at index {index}: {str(e)}")
            skipped_reviews += 1  # Track errors as skipped reviews

    print(f"Total predictions recorded: {len(predictions)}")
    print(f"Total actual sentiments recorded: {len(actual)}")
    print(f"Total reviews skipped: {skipped_reviews}")

    return predictions, actual, misclassified_reviews

# Call the function to get predictions and actual labels
predictions, actual, misclassified_reviews = predict_sentiment(sampled_df, model_id, client)

# Now you can evaluate the predictions using classification metrics
print("\nClassification Report:")
print(classification_report(actual, predictions))

# Generate the confusion matrix
cm = confusion_matrix(actual, predictions, labels=['positive', 'negative'])
print("\nConfusion Matrix:")
print(cm)

# Print out the misclassified reviews
print("\nMisclassified Reviews:")
for review in misclassified_reviews:
    print(f"Index: {review['index']}, Review: {review['review']}")
    print(f"Predicted: {review['predicted_label']}, Actual: {review['actual_label']}\n")

Total reviews in dataset: 100


Predicting Sentiments: 100%|██████████| 100/100 [00:40<00:00,  2.48it/s]

Total predictions recorded: 100
Total actual sentiments recorded: 100
Total reviews skipped: 0

Classification Report:
              precision    recall  f1-score   support

    negative       0.96      0.94      0.95        50
    positive       0.94      0.96      0.95        50

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100


Confusion Matrix:
[[48  2]
 [ 3 47]]

Misclassified Reviews:
Index: 8, Review: You have to know that you can't get the Bento Box after 4 pm. Very sad about this glitch. We came all the way from NW Tucson/Marana for it.
Predicted: negative, Actual: positive

Index: 16, Review: The flavor for the Pad Thai tastes plain, but I'm sure if you want some spice they would change it for you.
Predicted: positive, Actual: negative

Index: 17, Review: A very unique place, the inside is eclectic and very cozy and charming.  This is Louisiana food at its best.  

**Code Cell 4** - Perform Sentiment Analysis Using Few-Shot Learning

In [62]:
import json
import random
import re
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

def get_n_shot_prompt(n_shot_pos_reviews, n_shot_neg_reviews):
    prompt = (
        "Classify the sentiment of the review strictly as either 'positive' or 'negative'.\n"
        "1. A positive review expresses satisfaction, approval, or favorable experiences.\n"
        "2. A negative review expresses dissatisfaction, disapproval, or unfavorable experiences.\n"
        "You are only allowed to output 'positive' or 'negative'. No extra explanation or information.\n"
        f"Example 1:\nReview: {n_shot_pos_reviews[0]}\nSentiment: positive\n\n"
        f"Example 2:\nReview: {n_shot_pos_reviews[1]}\nSentiment: positive\n\n"
        f"Example 3:\nReview: {n_shot_neg_reviews[0]}\nSentiment: negative\n\n"
        f"Example 4:\nReview: {n_shot_neg_reviews[1]}\nSentiment: negative\n\n"
        "Now, classify the sentiment of the following review:\n"
        "Review: {{review}}\n"
        "Sentiment: "
    )
    return prompt

def predict_sentiment_with_n_shot(sampled_df, model_id, client):
    predictions = []
    actual = []
    skipped_rows = 0
    missing_predictions = []
    misclassified_reviews = []

    print(f"Total reviews in dataset: {sampled_df.shape[0]}")

    # Randomly sample 2 positive and 2 negative reviews
    pos_reviews = sampled_df[sampled_df['Sentiment_Label'].str.lower() == 'positive'].sample(2, random_state=42)['text'].tolist()
    neg_reviews = sampled_df[sampled_df['Sentiment_Label'].str.lower() == 'negative'].sample(2, random_state=42)['text'].tolist()

    # Get the N-shot prompt with the selected reviews
    prompt = get_n_shot_prompt(pos_reviews, neg_reviews)

    # Loop through the DataFrame and perform sentiment prediction for each review
    for index, row in tqdm(sampled_df.iterrows(), total=sampled_df.shape[0], desc="Predicting Sentiments", leave=False):
        review = row.get('text', '').strip()  # Handle missing review text gracefully
        actual_sentiment = row.get('Sentiment_Label', '').strip().lower()  # Handle missing sentiment label gracefully

        # Skip rows with missing review text or sentiment label
        if not review or not actual_sentiment or actual_sentiment not in ['positive', 'negative']:
            skipped_rows += 1
            continue
        
        # Format the N-shot prompt for the current review
        review_prompt = prompt.replace("{{review}}", review)
        
        native_request = {
            "anthropic_version": "bedrock-2023-05-31",  # Specify the version of the model
            "max_tokens": 20,  # Limit the number of tokens in the response
            "temperature": 0.4,  # Set the temperature (creativity level)
            "messages": [
                {"role": "user", "content": review_prompt}  # Provide the prompt to the model
            ]
        }
        
        # Convert request to JSON
        request = json.dumps(native_request)
        
        try:
            # Send the request to the model and receive the response
            response = client.invoke_model(modelId=model_id, body=request)
            
            # Parse the response body
            response_body = response["body"].read()
            if response_body:
                model_response = json.loads(response_body)

                # Extract the sentiment prediction from the model response
                raw_prediction = model_response.get("content", [""])[0].get("text", "").strip().lower()

                # Use regex to extract only 'positive' or 'negative' if extra text is present
                match = re.search(r"\b(positive|negative)\b", raw_prediction)
                prediction = match.group(1) if match else None
                
                # Store valid predictions only
                if prediction in ['positive', 'negative']:
                    predictions.append(prediction)
                    actual.append(actual_sentiment)

                    # Identify misclassified reviews
                    if prediction != actual_sentiment:
                        misclassified_reviews.append({
                            'index': index,
                            'review': review,
                            'actual': actual_sentiment,
                            'predicted': prediction
                        })
                else:
                    missing_predictions.append(index)  # Log the index of rows with invalid predictions

        except Exception as e:
            print(f"Error during model invocation at index {index}: {str(e)}")
    
    # Report results
    print(f"\nTotal predictions recorded: {len(predictions)}")
    print(f"Total actual sentiments recorded: {len(actual)}")
    print(f"Total skipped rows: {skipped_rows}")
    print(f"Total missing predictions at indices: {missing_predictions}\n")

    return predictions, actual, misclassified_reviews

# Call the function to get predictions and actual labels
predictions, actual, misclassified_reviews = predict_sentiment_with_n_shot(sampled_df, model_id, client)

# Print the classification report
print("Classification Report:")
print(classification_report(actual, predictions))

# Generate the confusion matrix
cm = confusion_matrix(actual, predictions, labels=['positive', 'negative'])
print("\nConfusion Matrix:")
print(cm)

# Print misclassified reviews
print("\nMisclassified Reviews:")
for misclassified in misclassified_reviews:
    print(f"Index: {misclassified['index']}, Review: {misclassified['review']}, Actual: {misclassified['actual']}, Predicted: {misclassified['predicted']}")

Total reviews in dataset: 100



Total predictions recorded: 100
Total actual sentiments recorded: 100
Total skipped rows: 0
Total missing predictions at indices: []

Classification Report:
              precision    recall  f1-score   support

    negative       0.94      0.94      0.94        50
    positive       0.94      0.94      0.94        50

    accuracy                           0.94       100
   macro avg       0.94      0.94      0.94       100
weighted avg       0.94      0.94      0.94       100


Confusion Matrix:
[[47  3]
 [ 3 47]]

Misclassified Reviews:
Index: 8, Review: You have to know that you can't get the Bento Box after 4 pm. Very sad about this glitch. We came all the way from NW Tucson/Marana for it., Actual: positive, Predicted: negative
Index: 17, Review: A very unique place, the inside is eclectic and very cozy and charming.  This is Louisiana food at its best.  The service was quite good and responsible.  We enjoyed the food very much, although some was a little Louisiana spicy.  Good e

**Code cell 5**

In [52]:

model_id_1='anthropic.claude-3-sonnet-20240229-v1:0' # LLM1

model_id_2='meta.llama3-70b-instruct-v1:0' # LLM2


In [55]:
random_index=random.randint(1,100) # create a random index

data_sample=sampled_df.iloc[random_index] # access a random sample from the dataset using random_index

data_sample

text               We have been going to this place since it open...
Sentiment_Label                                             positive
Name: 33, dtype: object

In [57]:
review=data_sample['text']

sentiment=data_sample['Sentiment_Label']

LLM 1

In [58]:
prompt=f'''
        Review: {review}
        
        summarize the given review.
        
        '''


native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "temperature": 0.5,
        "messages": [
            {"role": "user", "content": prompt}
        ]
}
request = json.dumps(native_request)  # convert the request into a json request

response= client.invoke_model(modelId=model_id_1,body=request) # passing the json request and calling the model

# the next two lines of code include parsing the model's response

model_response= json.loads(response["body"].read()) 

prediction = model_response.get("content", [""])[0].get("text", "") 

print(prediction)

Here is a summary of the review:

The review is very positive about the restaurant. The reviewer has been going there since it opened and has always had excellent food and service. They highly recommend the Wildcat burger and the "NelsonLove" appetizer plate as a main course, describing it as fantastic. The reviewer encourages others to visit this restaurant.


LLM 2

In [59]:
prompt=f'''
        Summary : {prediction}
        Now from the summary, list the things the reviewer likes and dislikes
        '''

# Note that, the syntax here for Llama differs from that of Claude
formatted_prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|> """
native_request = {
    "prompt": formatted_prompt,
    "max_gen_len": 512,
    "temperature": 0.5,
}
request = json.dumps(native_request) # converting into json request

response = client.invoke_model(modelId=model_id_2, body=request) # invoking the model

# parsing the model's response

model_response = json.loads(response["body"].read())  

output = model_response.get("generation", "")   # Note that, LLaMA (Meta) Response Formats here differs from that of Claude (Anthropic)

print(output)

<|end_header_id|><|start_header_id|>

Based on the summary, here are the things the reviewer likes and dislikes:

**Likes:**

* The restaurant in general
* The food at the restaurant
* The service at the restaurant
* The Wildcat burger
* The "NelsonLove" appetizer plate as a main course

**Dislikes:**

* None mentioned


**Text cell 6**

1. Comparing the performance for zero shot and few-shot learning

With the existing sentiment label 0 for negative and 1 for positive in the available dataset, the zero-shot learning model achieved an accuracy of 95%, with a high precision of 0.96 and recall of 0.94 for negative reviews, and 0.94 precision and 0.96 recall for positive reviews. The confusion matrix for zero-shot showed 48 true positives, 2 false positives, 3 false negatives, and 47 true negatives, indicating a well-balanced model performance.

On the other hand, the few-shot learning model, trained with 2 positive and 2 negative examples, achieved a slightly lower accuracy of 93%. Its precision for negative reviews was 0.92, and recall was 0.94, while for positive reviews, precision was 0.94 and recall was 0.92. The confusion matrix showed 46 true positives, 4 false positives, 3 false negatives, and 47 true negatives. While it performed well, it introduced more false positives and slightly lower recall for positive reviews compared to zero-shot learning, indicating a trade-off in performance due to the additional training examples.

2. Assess where LLM predictions differ from actual label                                                         
                                                                                                                 Zero-shot Prompting:

Based on the results from code cell 3  

False Positives (FP): The model incorrectly predicted positive sentiment for 2 reviews that were actually negative.

False Negatives (FN): The model incorrectly predicted negative sentiment for 3 reviews that were actually positive.

n-shot Prompting:

Based on the results from code cell 4           

False Positives (FP): The model incorrectly predicted positive sentiment for 4 reviews that were actually negative.

False Negatives (FN): The model incorrectly predicted negative sentiment for 3 reviews that were actually positive 
                                                                                                                 

3. Analyze potential reasons for misclassifications

zero shot prompting

Based on the misclassified reviews above from code cell 3, the misclassifications occurred due to mixed sentiments in reviews, where both positive and negative aspects were present. The model also struggled with context, negations, and subtle language nuances. In some cases, it focused too much on specific features, misinterpreting the overall sentiment.                                                                                                                                                                                          n-

n shot prompting                                                                                                 
                                                              
Based on the misclassified reviews from code cell 4, the misclassifications occurred due to mixed sentiments in the reviews, where positive aspects were overshadowed by dissatisfaction in certain areas. For example, reviews with complaints about service or food quality were misclassified as negative despite praising other elements like the ambiance or specific dishes. The model struggled to balance the positive and negative sentiments, leading to errors in classification. Additionally, complex reviews with both praise and critique likely caused confusion in determining the overall sentiment. 

Both the model missclassified reviews with index, 8 and 17.
   
4. Differences in outputs from various LLMs

Claude provides a narrative summary of the review, highlighting the positive sentiment, specific recommendations, and the reviewer’s loyalty to the restaurant. Lama, on the other hand, presents the review in a structured, bullet-point format, focusing on what the reviewer liked without adding extra context or emotions. Claude's output is more detailed and conversational, while Lama’s is straightforward and factual.                                                                                                   

**Text cell 7**: Acknowledgement

> Used Chatgpt
    
> Referred modules code, Sentiment-Analysis_with_prompting_techniques